In [1]:
def meanf (arg1): # Array of numbers to be passed
    if len(arg1) == 0:
        return("Array is empty")
    sumarg1 = 0
    for number in range(len(arg1)):
        sumarg1 += arg1[number]
        Avg = sumarg1/len(arg1)
    return(Avg)

In [2]:
def varf (arg1): # Array of numbers to be passed
    if len(arg1) == 0:
        return("Array is empty")
    meanv = meanf(arg1)
    sqdsumdiff = 0
    for number in range(len(arg1)):
        sqdsumdiff += (arg1[number]-meanv)*(arg1[number]-meanv)
        var = sqdsumdiff/(len(arg1)-1)
    return (meanv,var) 
 # unpack this output tuple for extracting meanv and Var after storing this in any variable eg. z (Mean, Variance) = z

In [3]:
#Building a dataframe from original, where values are identified as invalid and a RC list created.

def BldDefDF(arg1,arg2,arg3) : 
    # arg1 df; 
    # arg2 list of RCs  of original DF where data is identified as invalid or NaN
    # arg3 is list of Col names


    RLst = []                       # list containing 'R' values from list of RCs list
    for pshin in range(len(arg2)) :
        RLst.append(arg2[pshin][0])

    Compact_list = []               # After Removing Duplicates from Rlist
    for num in RLst:
        if num not in Compact_list: 
            Compact_list.append(num) 
    Compact_list.sort()
  
    DFDef = pd.DataFrame(columns = arg3)
    for itno in range(len(Compact_list)) :
        DFDef = DFDef.append(arg1.iloc[Compact_list[itno]])

    return(DFDef,Compact_list)

In [4]:
# for getting row and columns details of each NAN

def Nandet(arg1,arg2,arg3) :  # df1,ColNames1;LenCol1
    
    
    TotNan = 0
    TotRC = []
    for ColNo in range(arg3) :
        ColSel = arg2[ColNo]
        NulRws  = (arg1[arg1[ColSel].isnull()].index.tolist())  #get Row nos for Nan in that Column
        NRLen = len(NulRws)
        for NoNul in range(NRLen) :
            TotNan += 1
            TotRC.append([NulRws[NoNul],ColNo])              #Create RC for the Nans in that Column
    #print(totNan,totRC)
    #input("enter")
    return(TotNan,TotRC)

In [5]:
# checking for invalid entries(other than NaNs) in text and numeric columns 

def DataValid (arg1,arg2,arg3,arg4,arg5): # df,LenDf,ColNames,ColTypes,LenCol
     
    df1        = arg1   
    LenDf1     = arg2
    ColNames1  = arg3
    ColTyp     = arg4
    LenCol1    = arg5

    Nopbl = "True"
    DelRws = "N"
    NaNRC = []
    PrblRCs =[]
    ColAvgVar =[[]]
    
    (TotNoNaN,TotRC) = Nandet(arg1,arg3,arg5)  # To get RC details of all NaNs.
    
    for noofcol in range(arg5) :
        ColValidNos = []
        NonStr = 0
        NonNmr = 0  
        NoofNaN = sum(pd.isnull(df1[ColNames1[noofcol]]))   # summing number of Nan in each column.
        for noofrw in range(arg2) :            
            if (ColTyp[noofcol] == "Text" and type(arg1.iloc[noofrw,noofcol]) != str ) :
                NonStr += 1
                TotRC.append([noofrw,noofcol])
                
            if ColTyp[noofcol] == "Numbers" :
                if type(df1.iloc[noofrw,noofcol]) != str :
                    ColValidNos.append(df1.iloc[noofrw,noofcol])
                if type(df1.iloc[noofrw,noofcol]) == str :
                    NonNmr += 1
                    TotRC.append([noofrw,noofcol])
                
        if ColTyp[noofcol] == "Numbers" :
            ColValidNos1 =[]
            for noc in range(len(ColValidNos)):
                if ~np.isnan(ColValidNos[noc]) :
                    ColValidNos1.append(ColValidNos[noc])
            ColAvgVar = varf(ColValidNos1)
            
        if (NonNmr != 0 or NonStr != 0 or NoofNaN != 0) :
            Nopbl = "False"
            print("Column ",ColNames[noofcol]," should have only ",ColTyp[noofcol]+" but also contains")
            if ColTyp[noofcol] == "Text" :
                msg = (str(NonStr)+" Number of Non Strings of which "+str(NoofNaN)+" are NaNs.")            
            if ColTyp[noofcol] == "Numbers" :
                msg = (str(NonNmr)+" Number of Non Numericals and "+str(NoofNaN)+" Number of NaNs.\nMean, Variance and Std Deviation of Valid entries in this column are "+"%.3f" %ColAvgVar[0]+"  "+"%.3f" %ColAvgVar[1]+" "+"%.3f" %math.sqrt(ColAvgVar[1]))

            print(msg,'\n')
            
    if Nopbl == "False" :
        (DFDefect,Dellst) = BldDefDF(arg1,TotRC,arg3)  #Building a dataframe having where values are identified as invalid.
        print("Total Number of NaNs are :",TotNoNaN,'.  Also check for other wrong entries as mentioned above. \n')
        print("Following is the list of rows where data is not valid and has to be corrected.\nNo of defective rows presented",len(DFDefect),"of ",LenDf1,"which is ","%.2f" %(len(DFDefect)*100/LenDf1),"%.\n\n")
        print(DFDefect)
        print("\nOnly defective data presented above. Complete including defective data given below, which could guide in rectifying\n")
        print(df1)
        DelRws = input("Please go throuh the above data. Enter y to delete rows with defective data and proceed to prepare MongoDB. \nWARNING !!!!!  Above mentioned % of data will be deleted !!!!!\n")    
        
    if DelRws.upper() == "Y" :
        df1 = df1.drop(Dellst,axis = 0)
        Nopbl = "True"
        
    return(Nopbl,df1)

In [6]:
# 15% rows checked for identifying type of data in columns. 

def IdenColTyp(arg1,arg2,arg3) : 
  
    # arg1 is df
    # arg2 is LenDf
    # arg3 is LenCol
    #15% of rows checked but what if length > 10000  
    
    if ((round(15/100)*arg2)%2) == 0 :          # To ensure odd number of rows selected for checking.
        QntytoChk = (round(15/100)*arg2)+1
    else :
        QntytoChk = (round(15/100)*arg2)
        
    ColTyp = []
    ColAvg = []
    for NoofCol in range(arg3) :
        NoofRwChkd = 0
        NoofStr = 0
        NoofNmr = 0
        randomlist = random.sample(range(0, arg2), QntytoChk)
        while NoofRwChkd < QntytoChk :
            RwNo  = randomlist[NoofRwChkd]             
            if type(arg1.iloc[RwNo,NoofCol]) == str :
                NoofStr += 1
            else : 
                NoofNmr += 1
            NoofRwChkd += 1
        if NoofStr > NoofNmr :
            ColTyp.append("Text")
        else :
            ColTyp.append("Numbers")
            
    return(ColTyp)

In [7]:
# checking for availability of file containing data to be analysed.

def FilChk() :
    print("\n\n File will be checked in dir",os.getcwd())
    Contin = "Y"
    while Contin == "Y" :
        nfile = input ("\n Enter File Name with extension:")
        Flnm = os.getcwd()+"\\"+nfile
        if os.path.isfile(Flnm) :
            print ("File",Flnm,"is available \n")
            Contin = "N"
        else :
            print ("File",Flnm,"is not available\n")
            Quit = input("Enter Q  to quit").upper()
            if Quit == "Q" :
                sys.exit('Quit on user input.')
    return(Flnm)

In [8]:
#changing working directory

def ChngDir() :
    print("Current Working Directory " , os.getcwd())
    Contin = "Y"    
    while Contin == "Y" :
        nwdir = input ("Enter Directory :")
        if os.path.exists(nwdir) :
            os.chdir(nwdir)
            print("Directory changed to ",os.getcwd())
            nopbl = "No"
            Contin = "N"
        else:
            print(" Working Directory ",nwdir,"  does not exist.")
            Quit = input("Enter Q  to quit").upper()
            if Quit == "Q" :
                sys.exit('Quit on user input')

In [9]:
#source to mongod

#%whos 

import pandas as pd
import numpy as np
import pymongo
import json
import csv
import os
import random
import sys
import math

ChngDir()                          #To change to directory containing working file

TargetFile = FilChk()              # To get file name from user

if TargetFile[-3:].upper() == "CSV":
    df =pd.read_csv(TargetFile)
    
elif TargetFile[-3:].upper() == "TXT" :
    df = pd.read_csv(TargetFile,delimiter=',')                        # HOW TO TAKE CARE OF OTHER DELIMITERS.

elif TargetFile[-3:].upper() == "XLS" or TargetFile[-4:].upper() == "XLSX"  :
    ShtName = input ("\n Enter Sheet name. If blank first sheet will taken:")
    if len(ShtName) == 0 :
        df = pd.read_excel(TargetFile)
    else :
        df = pd.read_excel(TargetFile,sheet_name = ShtName)
else :
    input("not a suitable file")
    # exit()                                                            # how to break out.


LenDf = len(df)

ColNames  = []
for Col in df.columns: 
    ColNames.append(Col)
LenCol = len(ColNames)

ColTypes = IdenColTyp(df,LenDf,LenCol)  # for finding out type of data column wise.

print("\nThe  columns in database are  : ", ColNames)     
print("The corresponding types are   : ", ColTypes)
print("Total no of rows are          :  "+str(LenDf)+". \n")

(proceed,FinDf) = DataValid(df,LenDf,ColNames,ColTypes,LenCol)      #  function for validating data

FinDf.reset_index(inplace = True) 

MongoDbreq = "N"

MongoDbreq = input ("\n\nDo you want to create MongoDb  ")

if proceed == "True" and MongoDbreq.upper() == "Y" :  

    myclient = pymongo.MongoClient("mongodb://localhost:27017/")

    LinRegDB = myclient["LinReg"]            # DB for a group
    LinRegIp = LinRegDB["LinReginput"]       # Collection 1 input file
    LinREgOp = LinRegDB["LinRegoutput"]      # Collection 2 Results. (single or multiple for different groups)

    delinp  = LinRegIp.delete_many({})       #  to ensure collectible is empty
    delout =  LinREgOp.delete_many({})       #  to ensure collectible is empty

    LenFinDf = len(FinDf)


    for noofrw in range(LenFinDf) :
        rw = '{"_id" : ' + str(noofrw+1)
        for noofcol in range(LenCol) :
            rw = rw+', "'+ColNames[noofcol]+'": '

            if type(FinDf.iloc[noofrw,noofcol]) == str :
                rw = rw+'"'+(FinDf.iloc[noofrw,noofcol])+'"'    #  to put quotes if data is a string
            else :                                        
                rw = rw+str(FinDf.iloc[noofrw,noofcol])         # not putting quotes if not a string
        rw = rw+' }'
        rwdict = json.loads(rw)
        x = LinRegIp.insert_one(rwdict)



    for x in LinRegIp.find():
        print(x)
else :
    print("\nMongodB not created")
    print("\nFollowing Data frames available")
    print("FinDf is the cleaned data base")
    %whos DataFrame
    print("\n\nFinDf is the cleaned data base")
        
print("\n\n\nData module completed")
    
#del TargetFile, inject, Col ,rw           # strings
#del rwdict,x                              # dict
#del ColNames, ColTypes                    # lists
#del LenCol, LenDf, noofcol, noofrw        # int
#del proceed                               # bool
# del BldDefDF, ChngDir, DataValid, FilChk, IdenColTyp, Modify_df, Nandet, meanf, varf  # funct

Current Working Directory  C:\Users\Manjunatha-Think
Enter Directory :C:\Users\Manjunatha-Think
Directory changed to  C:\Users\Manjunatha-Think


 File will be checked in dir C:\Users\Manjunatha-Think

 Enter File Name with extension:Automobiles_price_mileage_country.csv
File C:\Users\Manjunatha-Think\Automobiles_price_mileage_country.csv is available 


The  columns in database are  :  ['Make', 'Country', 'Mileage', 'Price']
The corresponding types are   :  ['Text', 'Text', 'Numbers', 'Numbers']
Total no of rows are          :  45. 



Do you want to create MongoDb  n

MongodB not created

Following Data frames available
FinDf is the cleaned data base
Variable   Type         Data/Info
---------------------------------
FinDf      DataFrame        index                <...>4   Japan       35   6488
df         DataFrame        index                <...>4   Japan       35   6488


FinDf is the cleaned data base



Data module completed


In [28]:
FinDf

,index,ID,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,target
0,1,2,0.005861,0.39858,0.19768,1.93900,9.5771,0.000000,0.007237,1.50890,...,0.269720,0.000000,0.79303,0.312710,6.2213,4.2785,78.499,4.6497,1.65390,0
1,2,3,0.110600,0.16117,0.47894,7.57110,263.9000,0.000000,0.138360,5.20450,...,0.159870,0.131850,0.80748,0.102340,9.4499,3.3443,38.867,9.3910,1.52720,0
2,3,4,0.063911,1.40730,-0.29595,0.52910,-46.2870,-0.714200,0.079071,-0.33076,...,-0.015327,-0.137300,1.01530,-1.673200,29.2710,14.1410,73.458,4.9688,4.67850,0
3,4,5,0.131680,0.66031,0.44121,2.70980,-23.5960,-0.200070,0.131910,0.51445,...,0.110170,0.387640,0.93881,1.184200,4.5462,15.1400,44.101,8.2764,7.10140,0
4,10,11,0.023758,0.46625,-0.00242,0.99133,-42.5230,-0.038259,0.023758,1.11220,...,0.033089,0.045815,0.96691,0.360890,10.0860,12.3020,69.646,5.2408,2.02220,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14287,31260,31261,0.144940,0.41312,0.43934,2.17040,84.5740,0.311570,0.180440,1.32660,...,0.104030,0.264470,0.89597,0.068882,18.4580,3.9967,85.033,4.2924,8.69610,0
14288,31261,31262,0.061518,0.20800,0.35507,3.16180,75.8590,0.096039,0.077805,3.80110,...,0.371600,0.000000,0.62854,0.055341,5.0126,3.2761,66.041,5.5269,1.88850,0
14289,31267,31268,0.006271,0.55712,-0.18681,0.49995,-150.7600,-0.049339,0.011349,0.73448,...,0.120700,0.015324,0.87930,0.448550,16.4810,5.2665,192.900,1.8922,0.86922,0
14290,31271,31272,0.064759,0.64625,0.14655,1.24290,636.1700,0.000000,0.077580,0.54739,...,0.009710,0.183060,0.99424,0.120300,141.1600,10.6630,29.916,12.2010,29.43700,0


In [21]:
FinDf = FinDf.reset_index(inplace = True) 

In [22]:
FinDf

In [24]:
FinDf